⚠️ This project is mandatory for certification bloc #1.

![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 


## Company's description 📇

<a href='https://www.kayak.com' target='_blank'>Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href='https://www.bookingholdings.com/' target='_blank'>Booking Holdings</a> which now holds: 

* <a href='https://booking.com/' target='_blank'>Booking.com</a>
* <a href='https://kayak.com/' target='_blank'>Kayak</a>
* <a href='https://www.priceline.com/' target='_blank'>Priceline</a>
* <a href='https://www.agoda.com/' target='_blank'>Agoda</a>
* <a href='https://Rentalcars.com/' target='_blank'>RentalCars</a>
* <a href='https://www.opentable.com/' target='_blank'>OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href='https://one-week-in.com/35-cities-to-visit-in-france/' target='_blank'>One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
['Mont Saint Michel',
'St Malo',
'Bayeux',
'Le Havre',
'Rouen',
'Paris',
'Amiens',
'Lille',
'Strasbourg',
'Chateau du Haut Koenigsbourg',
'Colmar',
'Eguisheim',
'Besancon',
'Dijon',
'Annecy',
'Grenoble',
'Lyon',
'Gorges du Verdon',
'Bormes les Mimosas',
'Cassis',
'Marseille',
'Aix en Provence',
'Avignon',
'Uzes',
'Nimes',
'Aigues Mortes',
'Saintes Maries de la mer',
'Collioure',
'Carcassonne',
'Ariege',
'Toulouse',
'Montauban',
'Biarritz',
'Bayonne',
'La Rochelle']
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

# Let's start

In [1]:
# Import libs:

import pandas as pd
import requests
import numpy as np
import plotly.express as px
import scrapy
from scrapy.crawler import CrawlerProcess
import logging
import os
from bs4 import BeautifulSoup
import time
import boto3
import psycopg2
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker


## Get weather data with an API

In [2]:
# Import list of cities

villes = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle",
]


In [3]:
# Creating a dataframe from the list of cities
df_cities = pd.DataFrame(data=villes, columns=["ville"])
df_cities = df_cities.set_index("ville")
df_cities = df_cities.reindex(villes)
df_cities = df_cities.reset_index()
df_cities = df_cities.rename(columns={"index": "id"})

df_cities


,ville
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen
5,Paris
6,Amiens
7,Lille
8,Strasbourg
9,Chateau du Haut Koenigsbourg


### Nominatim API : get the gps coordinates of all the cities => https://nominatim.org/release-docs/latest/

In [4]:
# Try a city with blank space in name to see if formatting needed:

params = {"city": "La Rochelle", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


[{'place_id': 281822562,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 117858,
  'boundingbox': ['46.1331804', '46.1908971', '-1.2419231', '-1.111097'],
  'lat': '46.1591126',
  'lon': '-1.1520434',
  'display_name': 'La Rochelle, Charente-Maritime, Nouvelle-Aquitaine, France métropolitaine, 17000, France',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.9114837096874572,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'},
 {'place_id': 282096286,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 1215878,
  'boundingbox': ['47.7388479', '47.7647325', '5.7060641', '5.7454734'],
  'lat': '47.75173925',
  'lon': '5.7245822865532014',
  'display_name': 'La Rochelle, Vesoul, Haute-Saône, Bourgogne-Franche-Comté, France métropolitaine, 70120, France',
  'class': 'boundary',

In [5]:
# I don't know 'Chateau du Haut Koenigsbourg', check if Nominatim knows it

params = {"city": "Chateau du Haut Koenigsbourg", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


[{'place_id': 49750339,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 4245068168,
  'boundingbox': ['48.2494726', '48.2495726', '7.3454423', '7.3455423'],
  'lat': '48.2495226',
  'lon': '7.3454923',
  'display_name': 'Château du Haut-Kœnigsbourg, Orschwiller, Sélestat-Erstein, Bas-Rhin, Grand Est, France métropolitaine, 67600, France',
  'class': 'place',
  'type': 'isolated_dwelling',
  'importance': 0.51}]

In [6]:
# 'Gorges du Verdon' isn't a city but a canyon, check if Nominatim works with or not

params = {"city": "Gorges du Verdon", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r

# Doesnt' works... Google search : 'Castellane' will be use for Gorges du Verdon


[]

Ariege is a department and not a city, its prefecture is 'Foix'. We'll use this city

#### Request API

In [7]:
# Creating a copy of df_cities to store the coordinates from Nominatim API

df_gps = df_cities.copy(deep=True)
lat_list = []
lon_list = []

for i in villes:
    print(f"Request for city: {i}")
    params = {"city": i, "country": "France", "format": "json"}
    # No 'Gorges du Verdon' city, replacing by 'Castellane'
    if i == "Gorges du Verdon":
        i = "Castellane"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    # 'Ariege' not a city, using the prefecture instead -> 'Foix'
    elif i == "Ariege":
        i = "Foix"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    else:
        r = requests.get(f"https://nominatim.openstreetmap.org/search?", params).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])

# Adding the coordinates to the dataframe
df_gps["lat"] = lat_list
df_gps["lon"] = lon_list


Request for city: Mont Saint Michel
Request for city: St Malo
Request for city: Bayeux
Request for city: Le Havre
Request for city: Rouen
Request for city: Paris
Request for city: Amiens
Request for city: Lille
Request for city: Strasbourg
Request for city: Chateau du Haut Koenigsbourg
Request for city: Colmar
Request for city: Eguisheim
Request for city: Besancon
Request for city: Dijon
Request for city: Annecy
Request for city: Grenoble
Request for city: Lyon
Request for city: Gorges du Verdon
Request for city: Bormes les Mimosas
Request for city: Cassis
Request for city: Marseille
Request for city: Aix en Provence
Request for city: Avignon
Request for city: Uzes
Request for city: Nimes
Request for city: Aigues Mortes
Request for city: Saintes Maries de la mer
Request for city: Collioure
Request for city: Carcassonne
Request for city: Ariege
Request for city: Toulouse
Request for city: Montauban
Request for city: Biarritz
Request for city: Bayonne
Request for city: La Rochelle


In [8]:
df_gps.head()


,ville,lat,lon
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


In [9]:
df_gps.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ville   35 non-null     object
 1   lat     35 non-null     object
 2   lon     35 non-null     object
dtypes: object(3)
memory usage: 968.0+ bytes


### OpenWeather : get weather of the week => https://openweathermap.org

In [10]:
# try at St Malo
parameters = {
    "lat": 48.649518,
    "lon": -2.0260409,
    "exclude": "current,minutely,hourly",
    "units": "metric",
    "appid": "aa423e6694bf72625fe1fe31544949dc",
    "lang": "fr",
}

r = requests.get(
    "https://api.openweathermap.org/data/2.5/onecall?", params=parameters
).json()

r


{'lat': 48.6495,
 'lon': -2.026,
 'timezone': 'Europe/Paris',
 'timezone_offset': 7200,
 'daily': [{'dt': 1656244800,
   'sunrise': 1656216416,
   'sunset': 1656274484,
   'moonrise': 1656209040,
   'moonset': 1656267300,
   'moon_phase': 0.92,
   'temp': {'day': 18.89,
    'min': 9.91,
    'max': 19.71,
    'night': 13.82,
    'eve': 16.62,
    'morn': 10.49},
   'feels_like': {'day': 18.19, 'night': 13.27, 'eve': 16.03, 'morn': 9.94},
   'pressure': 1013,
   'humidity': 52,
   'dew_point': 8.31,
   'wind_speed': 6.25,
   'wind_deg': 207,
   'wind_gust': 10.29,
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'légère pluie',
     'icon': '10d'}],
   'clouds': 66,
   'pop': 0.4,
   'rain': 0.54,
   'uvi': 6.36},
  {'dt': 1656331200,
   'sunrise': 1656302841,
   'sunset': 1656360882,
   'moonrise': 1656297120,
   'moonset': 1656357600,
   'moon_phase': 0.95,
   'temp': {'day': 15.64,
    'min': 11.43,
    'max': 16.92,
    'night': 13.66,
    'eve': 15.74,
    'morn':

In [11]:
# Weather in 3 days at St Malo
r["daily"][3:]


[{'dt': 1656504000,
  'sunrise': 1656475700,
  'sunset': 1656533670,
  'moonrise': 1656474900,
  'moonset': 1656536820,
  'moon_phase': 0,
  'temp': {'day': 19.26,
   'min': 14.53,
   'max': 19.76,
   'night': 14.87,
   'eve': 15.58,
   'morn': 14.53},
  'feels_like': {'day': 18.65, 'night': 14.42, 'eve': 15.2, 'morn': 14.21},
  'pressure': 1013,
  'humidity': 54,
  'dew_point': 9.18,
  'wind_speed': 6.19,
  'wind_deg': 197,
  'wind_gust': 11.58,
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'couvert',
    'icon': '04d'}],
  'clouds': 94,
  'pop': 0.09,
  'uvi': 6.64},
 {'dt': 1656590400,
  'sunrise': 1656562132,
  'sunset': 1656620059,
  'moonrise': 1656564600,
  'moonset': 1656625620,
  'moon_phase': 0.04,
  'temp': {'day': 16.62,
   'min': 13.03,
   'max': 16.62,
   'night': 14.04,
   'eve': 14.89,
   'morn': 13.49},
  'feels_like': {'day': 16.09, 'night': 13.48, 'eve': 14.45, 'morn': 13.01},
  'pressure': 1013,
  'humidity': 67,
  'dew_point': 9.9,
  'wind_spee

In [12]:
day3 = r["daily"][3]  # Weather in 3 days

# desciption of the main weather in 3 days at St Malo
day3["weather"][0]["main"]


'Clouds'

In [13]:
df_gps_weather = df_gps.copy(deep=True)
my_api_key = "aa423e6694bf72625fe1fe31544949dc"

temperatures_list = []
rain_list = []
weather_list = []

days = list(range(1, 8))

for i in df_gps_weather.itertuples():
    lat = i.lat
    lon = i.lon

    parameters = {
        "lat": {lat},
        "lon": {lon},
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": "aa423e6694bf72625fe1fe31544949dc",
        "lang": "fr",
    }

    r = requests.get(
        f"https://api.openweathermap.org/data/3.0/onecall?", parameters
    ).json()
    forecast_7days = r["daily"][1:]  # Getting the weather data for the next 7 days
    temperatures = [int(d["feels_like"]["day"]) for d in forecast_7days]
    rain = [int(d["pop"] * 100) for d in forecast_7days]
    weather = [str(d["weather"][0]["main"]) for d in forecast_7days]
    temperatures_list.append(temperatures)
    rain_list.append(rain)
    weather_list.append(weather)

df_gps_weather["jour_+x"] = [days for _ in range(len(df_gps_weather))]
df_gps_weather["temperature_ressentie"] = temperatures_list
df_gps_weather["probabilite_de_pluie"] = rain_list
df_gps_weather["meteo_principale"] = weather_list
# df_weather['rang'] = df_weather['probabilite_de_pluie'].sort_values()
df_gps_weather["score"] = df_gps_weather.apply(
    lambda x: ((np.mean(x["temperature_ressentie"])))
    - (np.mean(x["probabilite_de_pluie"]) / 10),
    axis=1,
).astype(int)


In [14]:
df_gps_weather


,ville,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,score
0,Mont Saint Michel,48.6359541,-1.511459954959514,"[1, 2, 3, 4, 5, 6, 7]","[17, 20, 20, 17, 19, 21, 19]","[52, 0, 15, 62, 37, 0, 0]","[Rain, Clouds, Clouds, Rain, Rain, Clouds, Clear]",16
1,St Malo,48.649518,-2.0260409,"[1, 2, 3, 4, 5, 6, 7]","[15, 19, 18, 16, 16, 20, 17]","[47, 0, 9, 53, 48, 0, 0]","[Rain, Clouds, Clouds, Rain, Rain, Clouds, Clo...",15
2,Bayeux,49.2764624,-0.7024738,"[1, 2, 3, 4, 5, 6, 7]","[17, 20, 21, 14, 19, 22, 20]","[65, 0, 38, 71, 46, 0, 0]","[Rain, Clouds, Clouds, Rain, Rain, Clouds, Clo...",15
3,Le Havre,49.4938975,0.1079732,"[1, 2, 3, 4, 5, 6, 7]","[17, 18, 19, 14, 17, 18, 16]","[32, 0, 60, 98, 50, 0, 0]","[Clear, Clouds, Rain, Rain, Rain, Clouds, Clear]",13
4,Rouen,49.4404591,1.0939658,"[1, 2, 3, 4, 5, 6, 7]","[19, 22, 22, 16, 18, 21, 21]","[24, 0, 18, 100, 70, 0, 0]","[Clear, Clouds, Clouds, Rain, Rain, Clouds, Cl...",16
5,Paris,48.8588897,2.3200410217200766,"[1, 2, 3, 4, 5, 6, 7]","[21, 23, 24, 22, 20, 23, 24]","[2, 0, 0, 81, 61, 0, 0]","[Clouds, Clouds, Clouds, Rain, Rain, Clouds, C...",20
6,Amiens,49.8941708,2.2956951,"[1, 2, 3, 4, 5, 6, 7]","[20, 22, 23, 15, 18, 21, 21]","[37, 0, 0, 96, 86, 0, 0]","[Clouds, Clear, Clouds, Rain, Rain, Clouds, Cl...",16
7,Lille,50.6365654,3.0635282,"[1, 2, 3, 4, 5, 6, 7]","[20, 22, 24, 19, 20, 21, 22]","[57, 0, 0, 61, 94, 0, 0]","[Rain, Clear, Clouds, Rain, Rain, Clouds, Clouds]",18
8,Strasbourg,48.584614,7.7507127,"[1, 2, 3, 4, 5, 6, 7]","[19, 23, 23, 29, 15, 24, 28]","[100, 16, 20, 21, 100, 20, 0]","[Rain, Clouds, Clouds, Clear, Rain, Rain, Clear]",19
9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923,"[1, 2, 3, 4, 5, 6, 7]","[15, 20, 22, 26, 12, 21, 26]","[100, 2, 18, 60, 100, 20, 0]","[Rain, Clouds, Clouds, Clear, Rain, Rain, Clear]",16


In [15]:
df_gps_weather.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ville                  35 non-null     object
 1   lat                    35 non-null     object
 2   lon                    35 non-null     object
 3   jour_+x                35 non-null     object
 4   temperature_ressentie  35 non-null     object
 5   probabilite_de_pluie   35 non-null     object
 6   meteo_principale       35 non-null     object
 7   score                  35 non-null     int64 
dtypes: int64(1), object(7)
memory usage: 2.3+ KB


In [16]:
df_gps_weather[["lat", "lon"]] = df_gps_weather[["lat", "lon"]].astype(float)


### List of cities where the weather will be the nicest within the next 7 days

In [17]:
df_gps_weather.sort_values(by=["score"], ascending=False)


,ville,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,score
24,Nimes,43.837425,4.360069,"[1, 2, 3, 4, 5, 6, 7]","[29, 27, 29, 29, 28, 29, 34]","[11, 95, 0, 0, 1, 0, 0]","[Clouds, Rain, Clear, Clouds, Clear, Clear, Cl...",27
23,Uzes,44.012128,4.419672,"[1, 2, 3, 4, 5, 6, 7]","[28, 26, 30, 30, 27, 29, 34]","[15, 96, 0, 0, 1, 0, 0]","[Clouds, Rain, Clear, Clouds, Clear, Clear, Cl...",27
22,Avignon,43.949249,4.805901,"[1, 2, 3, 4, 5, 6, 7]","[29, 25, 29, 31, 27, 28, 34]","[15, 99, 0, 2, 0, 0, 0]","[Clouds, Rain, Clear, Clouds, Clear, Clear, Cl...",27
21,Aix en Provence,43.529842,5.447474,"[1, 2, 3, 4, 5, 6, 7]","[31, 28, 29, 29, 29, 29, 33]","[28, 76, 0, 40, 0, 0, 0]","[Clouds, Rain, Clear, Rain, Clear, Clear, Clear]",27
20,Marseille,43.296174,5.369953,"[1, 2, 3, 4, 5, 6, 7]","[29, 26, 28, 26, 28, 28, 30]","[12, 79, 0, 36, 0, 0, 0]","[Clouds, Rain, Clear, Rain, Clear, Clear, Clear]",26
19,Cassis,43.214036,5.539632,"[1, 2, 3, 4, 5, 6, 7]","[29, 26, 28, 26, 28, 28, 30]","[10, 67, 0, 45, 0, 0, 0]","[Clouds, Rain, Clear, Rain, Clear, Clear, Clear]",26
25,Aigues Mortes,43.565823,4.191284,"[1, 2, 3, 4, 5, 6, 7]","[28, 26, 27, 25, 27, 27, 30]","[7, 85, 0, 0, 1, 0, 0]","[Clouds, Rain, Clear, Clouds, Clear, Clear, Cl...",25
18,Bormes les Mimosas,43.157217,6.329254,"[1, 2, 3, 4, 5, 6, 7]","[29, 23, 29, 28, 27, 27, 29]","[3, 93, 0, 56, 0, 0, 0]","[Clear, Rain, Clear, Rain, Clouds, Clear, Clear]",25
26,Saintes Maries de la mer,43.452277,4.428717,"[1, 2, 3, 4, 5, 6, 7]","[27, 25, 26, 24, 27, 26, 30]","[1, 93, 0, 1, 0, 0, 0]","[Clouds, Rain, Clear, Clouds, Clear, Clear, Cl...",25
27,Collioure,42.525050,3.083155,"[1, 2, 3, 4, 5, 6, 7]","[26, 27, 28, 26, 24, 27, 28]","[36, 56, 0, 8, 0, 0, 0]","[Rain, Rain, Clear, Clouds, Clear, Clear, Clear]",25


### Save all the results in a `.csv` file

In [18]:
df_gps_weather = df_gps_weather.set_index("ville")
df_gps_weather = df_gps_weather.reindex(villes)
df_gps_weather = df_gps_weather.reset_index()
df_gps_weather = df_gps_weather.rename(columns={"index": "id"})
df_gps_weather


,ville,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,score
0,Mont Saint Michel,48.635954,-1.511460,"[1, 2, 3, 4, 5, 6, 7]","[17, 20, 20, 17, 19, 21, 19]","[52, 0, 15, 62, 37, 0, 0]","[Rain, Clouds, Clouds, Rain, Rain, Clouds, Clear]",16
1,St Malo,48.649518,-2.026041,"[1, 2, 3, 4, 5, 6, 7]","[15, 19, 18, 16, 16, 20, 17]","[47, 0, 9, 53, 48, 0, 0]","[Rain, Clouds, Clouds, Rain, Rain, Clouds, Clo...",15
2,Bayeux,49.276462,-0.702474,"[1, 2, 3, 4, 5, 6, 7]","[17, 20, 21, 14, 19, 22, 20]","[65, 0, 38, 71, 46, 0, 0]","[Rain, Clouds, Clouds, Rain, Rain, Clouds, Clo...",15
3,Le Havre,49.493898,0.107973,"[1, 2, 3, 4, 5, 6, 7]","[17, 18, 19, 14, 17, 18, 16]","[32, 0, 60, 98, 50, 0, 0]","[Clear, Clouds, Rain, Rain, Rain, Clouds, Clear]",13
4,Rouen,49.440459,1.093966,"[1, 2, 3, 4, 5, 6, 7]","[19, 22, 22, 16, 18, 21, 21]","[24, 0, 18, 100, 70, 0, 0]","[Clear, Clouds, Clouds, Rain, Rain, Clouds, Cl...",16
5,Paris,48.858890,2.320041,"[1, 2, 3, 4, 5, 6, 7]","[21, 23, 24, 22, 20, 23, 24]","[2, 0, 0, 81, 61, 0, 0]","[Clouds, Clouds, Clouds, Rain, Rain, Clouds, C...",20
6,Amiens,49.894171,2.295695,"[1, 2, 3, 4, 5, 6, 7]","[20, 22, 23, 15, 18, 21, 21]","[37, 0, 0, 96, 86, 0, 0]","[Clouds, Clear, Clouds, Rain, Rain, Clouds, Cl...",16
7,Lille,50.636565,3.063528,"[1, 2, 3, 4, 5, 6, 7]","[20, 22, 24, 19, 20, 21, 22]","[57, 0, 0, 61, 94, 0, 0]","[Rain, Clear, Clouds, Rain, Rain, Clouds, Clouds]",18
8,Strasbourg,48.584614,7.750713,"[1, 2, 3, 4, 5, 6, 7]","[19, 23, 23, 29, 15, 24, 28]","[100, 16, 20, 21, 100, 20, 0]","[Rain, Clouds, Clouds, Clear, Rain, Rain, Clear]",19
9,Chateau du Haut Koenigsbourg,48.249523,7.345492,"[1, 2, 3, 4, 5, 6, 7]","[15, 20, 22, 26, 12, 21, 26]","[100, 2, 18, 60, 100, 20, 0]","[Rain, Clouds, Clouds, Clear, Rain, Rain, Clear]",16


In [19]:
df_gps_weather.to_csv("df_gps_weather.csv", index=False)


### Plotly to display the best destinations on a map

In [20]:
df_plotly = df_gps_weather.apply(
    pd.Series.explode
)  # To obtain a line per day and per city

df_plotly[["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]] = df_plotly[
    ["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]
].astype(int)

fig = px.scatter_mapbox(
    df_plotly,
    lat="lat",
    lon="lon",
    hover_name="ville",
    zoom=4,
    hover_data=["meteo_principale", "probabilite_de_pluie", "temperature_ressentie"],
    color="temperature_ressentie",
    color_continuous_scale="thermal",
    mapbox_style="carto-positron",
)
fig.show()


## Scrape Booking.com 

### Get hotels and their URLs

In [21]:
class BookingSpider(scrapy.Spider):
    name = "Booking_data"
    cities = df_cities.ville
    start_urls = ["https://www.booking.com/index.fr.html"]

    def parse(self, response):
        for i in df_cities.ville:
            yield scrapy.FormRequest.from_response(
                response, formdata={"ss": i}, callback=self.after_search
            )

    # Important note: the classes names change often and may not be correct at a later date
    def after_search(self, response):
        cities = response.url.split("ss=")[-1].split("&")[0]

        booking = response.css(".d4924c9e74")

        for k in booking:

            yield {
                "ville": cities,
                "hotels": k.css("a div.fcab3ed991.a23c043802::text").getall(),
                "liens": k.css("h3.a4225678b2 a::attr(href)").getall(),
            }

        try:
            next_page = response.css("a.paging-next").attrib["href"]
        except KeyError:
            logging.info("No next page. Terminating crawling process.")
        else:
            yield response.follow(next_page, callback=self.after_search)


# Name of the file where the results will be saved
filename = "hotels.json"

# If file already exists, delete before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir():
    os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(
    settings={
        "USER_AGENT": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1",
        "LOG_LEVEL": logging.INFO,
        "FEEDS": {
            filename: {"format": "json"},
        },
        "AUTOTHROTTLE_ENABLED": True,
    }
)

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()


2022-06-26 06:45:20 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-06-26 06:45:20 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.12 (main, Apr  5 2022, 01:53:17) - [Clang 12.0.0 ], pyOpenSSL 21.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 3.4.8, Platform macOS-10.16-x86_64-i386-64bit
2022-06-26 06:45:20 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, '
               'like Gecko) Chrome/13.0.782.112 Safari/535.1'}
2022-06-26 06:45:20 [scrapy.extensions.telnet] INFO: Telnet Password: 992bce00d76fa398
2022-06-26 06:45:20 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extension

### Get hotels' coordinates

In [22]:
# Create a DataFrame from the json file

df = pd.read_json("hotels.json")
df["ville"] = df["ville"].str.replace("+", " ")
df


2022-06-26 06:45:47 [py.warnings] WARNING: /var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_2400/2552240652.py:4: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.




,ville,hotels,liens
0,Mont Saint Michel,"[Hôtel Vert, Le Relais Saint Michel, Hotel De ...",[https://www.booking.com/hotel/fr/vert.fr.html...
1,St Malo,"[Le Cargo by Cocoonr, Hotel d'Aleth, Hotel Ajo...",[https://www.booking.com/hotel/fr/le-cargo-sai...
2,Bayeux,"[Le Mogador, ibis budget Bayeux, Domaine de Ba...",[https://www.booking.com/hotel/fr/le-mogador-b...
3,Le Havre,"[Hilton Garden Inn Le Havre Centre, Holiday In...",[https://www.booking.com/hotel/fr/hilton-garde...
4,Rouen,"[Radisson Blu Hotel, Rouen Centre, Maison hype...",[https://www.booking.com/hotel/fr/radisson-blu...
5,Paris,"[Westside Arc de Triomphe Hotel, Grand Hôtel A...",[https://www.booking.com/hotel/fr/westside-arc...
6,Amiens,"[Holiday Inn Express Amiens, an IHG Hotel, Le ...",[https://www.booking.com/hotel/fr/express-by-h...
7,Lille,"[CALM Appart' & Hostel, Hotel Lille Europe, Mo...",[https://www.booking.com/hotel/fr/calm-apparth...
8,Strasbourg,"[City Residence Access Strasbourg, Comfort Hot...",[https://www.booking.com/hotel/fr/city-residen...
9,Chateau du Haut Koenigsbourg,"[Les Chambres du Haut-Koenigsbourg, Gîte L'Oré...",[https://www.booking.com/hotel/fr/les-chambres...


for i in range(len(df["ville"])):
    df["hotels"][i] = df["hotels"][i][0:5]
    df["liens"][i] = df["liens"][i][0:5]


In [23]:
# BeautifulSoup iterates on each URL to get the hotel's score, GPS coordinates and description.
df["lat"] = 0
df["lon"] = 0
df["description"] = "---"
df["score"] = 0.0

navigator = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1"

for i in range(len(df["liens"])):
    lat_list = []
    lon_list = []
    description_list = []
    score_list = []

    hotel_list = df["liens"][i]

    for i2 in hotel_list:

        # Sometimes BeautifulSoup doesn't manage to gather data. When it fails, it tries again.
        try:
            page = requests.get(i2, headers={"User-Agent": navigator})
            soup = BeautifulSoup(page.text, "html.parser")
        except:
            page = requests.get(i2, headers={"User-Agent": navigator})
            soup = BeautifulSoup(page.text, "html.parser")

        lat_list.append(
            soup.select("p.address.address_clean a")[0]
            .get("data-atlas-latlng")
            .split(",")[0]
        )
        lon_list.append(
            soup.select("p.address.address_clean a")[0]
            .get("data-atlas-latlng")
            .split(",")[1]
        )
        description_list.append(
            soup.select("div#property_description_content")[0].get_text()
        )

        try:
            score_list.append(soup.select("div.b5cd09854e.d10a6220b4")[0].get_text())

        except:
            # 2 hotels over the 700 that I am going to scrap dont have a score yet but I still need one for the visualization. I set it to 1.
            score_list.append("1.0")

        time.sleep(1.4)

    df["lat"][i] = lat_list
    df["lon"][i] = lon_list
    df["description"][i] = description_list
    df["score"][i] = score_list

    print(f"city {df['ville'].iloc[i]} : OK")


2022-06-26 06:47:24 [py.warnings] WARNING: /var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_2400/918478526.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2022-06-26 06:47:24 [py.warnings] WARNING: /var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_2400/918478526.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2022-06-26 06:47:24 [py.warnings] WARNING: /var/folders/hr/m4w2vfm12db4g050bkkr7y540000gn/T/ipykernel_2400/918478526.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

city Mont Saint Michel : OK
city St Malo : OK
city Bayeux : OK
city Le Havre : OK
city Rouen : OK
city Paris : OK
city Amiens : OK
city Lille : OK
city Strasbourg : OK
city Chateau du Haut Koenigsbourg : OK
city Colmar : OK
city Eguisheim : OK
city Besancon : OK
city Dijon : OK
city Annecy : OK
city Grenoble : OK
city La Rochelle : OK
city Bayonne : OK
city Biarritz : OK
city Montauban : OK
city Toulouse : OK
city Ariege : OK
city Carcassonne : OK
city Collioure : OK
city Saintes Maries de la mer : OK
city Aigues Mortes : OK
city Nimes : OK
city Uzes : OK
city Avignon : OK
city Aix en Provence : OK
city Marseille : OK
city Cassis : OK
city Bormes les Mimosas : OK
city Gorges du Verdon : OK
city Lyon : OK


In [24]:
df


,ville,hotels,liens,lat,lon,description,score
0,Mont Saint Michel,"[Hôtel Vert, Le Relais Saint Michel, Hotel De ...",[https://www.booking.com/hotel/fr/vert.fr.html...,"[48.61470049, 48.61758727, 48.61688155, 48.635...","[-1.50961697, -1.51039615, -1.51091784, -1.510...",[\nVous pouvez bénéficier d'une réduction Geni...,"[8,0, 7,9, 7,2, 7,3, 7,2, 8,2, 8,0, 7,2, 8,0, ..."
1,St Malo,"[Le Cargo by Cocoonr, Hotel d'Aleth, Hotel Ajo...",[https://www.booking.com/hotel/fr/le-cargo-sai...,"[48.64842800, 48.63593081, 48.64735692, 48.651...","[-2.02664200, -2.02171236, -2.02519655, -2.023...",[\nCet établissement est à 2 minutes à pied de...,"[8,3, 7,8, 8,5, 8,0, 7,4, 7,6, 8,3, 8,6, 7,3, ..."
2,Bayeux,"[Le Mogador, ibis budget Bayeux, Domaine de Ba...",[https://www.booking.com/hotel/fr/le-mogador-b...,"[49.27933593, 49.25424209, 49.27232560, 49.272...","[-0.70705610, -0.64648747, -0.69851010, -0.707...",[\nL'établissement Le Mogador est situé à Baye...,"[8,1, 8,2, 9,3, 8,6, 8,6, 7,9, 8,5, 8,1, 9,4, ..."
3,Le Havre,"[Hilton Garden Inn Le Havre Centre, Holiday In...",[https://www.booking.com/hotel/fr/hilton-garde...,"[49.49008699, 49.49823800, 49.49641693, 49.492...","[0.09774696, 0.12883700, 0.15151234, 0.1231080...",[\nLe Hilton Garden Inn Le Havre Centre est si...,"[8,8, 8,2, 7,6, 8,2, 8,5, 8,6, 8,4, 7,1, 8,1, ..."
4,Rouen,"[Radisson Blu Hotel, Rouen Centre, Maison hype...",[https://www.booking.com/hotel/fr/radisson-blu...,"[49.44644100, 49.44465180, 49.44117285, 49.437...","[1.09412000, 1.09132620, 1.09517545, 1.0972528...",[\nVous pouvez bénéficier d'une réduction Geni...,"[8,9, 8,7, 8,2, 8,4, 7,5, 8,6, 7,4, 8,5, 8,2, ..."
5,Paris,"[Westside Arc de Triomphe Hotel, Grand Hôtel A...",[https://www.booking.com/hotel/fr/westside-arc...,"[48.88129894, 48.85925680, 48.88141526, 48.876...","[2.29480304, 2.36840850, 2.32455321, 2.3269844...","[\nDoté d’un bar-salon, le Westside Arc de Tri...","[8,0, 8,1, 8,4, 8,1, 8,7, 8,7, 8,5, 8,5, 8,4, ..."
6,Amiens,"[Holiday Inn Express Amiens, an IHG Hotel, Le ...",[https://www.booking.com/hotel/fr/express-by-h...,"[49.89167620, 49.89538310, 49.89444519, 49.887...","[2.30775386, 2.29777200, 2.30094239, 2.3116109...","[\nRénové en avril 2015, le Holiday Inn Expres...","[8,1, 9,3, 8,3, 8,1, 8,0, 7,6, 9,5, 8,9, 8,3, ..."
7,Lille,"[CALM Appart' & Hostel, Hotel Lille Europe, Mo...",[https://www.booking.com/hotel/fr/calm-apparth...,"[50.63734618, 50.63788756, 50.62783100, 50.635...","[3.06945696, 3.07268500, 3.06359200, 3.0697072...",[\n\nVous pouvez bénéficier d'une réduction Ge...,"[8,1, 8,0, 8,6, 7,7, 8,0, 7,3, 7,8, 8,5, 8,0, ..."
8,Strasbourg,"[City Residence Access Strasbourg, Comfort Hot...",[https://www.booking.com/hotel/fr/city-residen...,"[48.59001400, 48.57269177, 48.57772068, 48.589...","[7.73987000, 7.72871353, 7.73397669, 7.7387986...","[\nSitué à Strasbourg, à moins de 900 mètres d...","[7,0, 8,3, 7,8, 7,1, 8,6, 9,0, 8,3, 8,2, 8,1, ..."
9,Chateau du Haut Koenigsbourg,"[Les Chambres du Haut-Koenigsbourg, Gîte L'Oré...",[https://www.booking.com/hotel/fr/les-chambres...,"[48.24702280, 48.24834892, 48.24840250, 48.180...","[7.34632093, 7.35484448, 7.35482570, 7.3092045...",[\n\nVous pouvez bénéficier d'une réduction Ge...,"[8,8, 9,5, 9,5, 9,5, 9,4, 9,0, 9,6, 9,4, 8,7, ..."


In [25]:
df = df.set_index("ville")
df = df.reindex(villes)
df = df.reset_index()
df = df.apply(pd.Series.explode)
df["meteo_score"] = df_gps_weather["score"]

df


,ville,hotels,liens,lat,lon,description,score,meteo_score
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,48.61470049,-1.50961697,\nVous pouvez bénéficier d'une réduction Geniu...,"8,0",16
0,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.61758727,-1.51039615,\n\nVous pouvez bénéficier d'une réduction Gen...,"7,9",16
0,Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.61688155,-1.51091784,\nCet établissement est à 1 minute à pied de l...,"7,2",16
0,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.63534943,-1.51037872,\nVous pouvez bénéficier d'une réduction Geniu...,"7,3",16
0,Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,48.61293783,-1.51010513,"\nNiché dans un écrin de verdure, à seulement ...","7,2",16
...,...,...,...,...,...,...,...,...
34,La Rochelle,Hôtel La Tour de Nesle La Rochelle Vieux Port,https://www.booking.com/hotel/fr/de-la-tour-de...,46.15809741,-1.15036190,\nVous pouvez bénéficier d'une réduction Geniu...,"8,1",15
34,La Rochelle,STUDIO EN HYPERCENTRE À 2 PAS DE L'HOTEL DE VI...,https://www.booking.com/hotel/fr/studio-en-hyp...,46.15983600,-1.15246920,\nLocated 800 metres from Concurrence and 2.4 ...,1.0,15
34,La Rochelle,"Hotel La Marine, Vieux Port",https://www.booking.com/hotel/fr/la-marine-la-...,46.15815872,-1.15267055,\nCet établissement est à 6 minutes à pied de ...,"8,5",15
34,La Rochelle,"Studio La Rochelle, 1 pièce, 2 personnes - FR-...",https://www.booking.com/hotel/fr/apartment-por...,46.13926000,-1.16084800,"\nLe Studio La Rochelle, 1 pièce, 2 personnes ...",1.0,15


In [26]:
df.to_csv("df_gps_hotels.csv", index=False)


### Test of CSV files

In [27]:
test_hotels = pd.read_csv("df_gps_hotels.csv")
test_hotels


,ville,hotels,liens,lat,lon,description,score,meteo_score
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,48.614700,-1.509617,\nVous pouvez bénéficier d'une réduction Geniu...,"8,0",16
1,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.617587,-1.510396,\n\nVous pouvez bénéficier d'une réduction Gen...,"7,9",16
2,Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.616882,-1.510918,\nCet établissement est à 1 minute à pied de l...,"7,2",16
3,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.635349,-1.510379,\nVous pouvez bénéficier d'une réduction Geniu...,"7,3",16
4,Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,48.612938,-1.510105,"\nNiché dans un écrin de verdure, à seulement ...","7,2",16
...,...,...,...,...,...,...,...,...
870,La Rochelle,Hôtel La Tour de Nesle La Rochelle Vieux Port,https://www.booking.com/hotel/fr/de-la-tour-de...,46.158097,-1.150362,\nVous pouvez bénéficier d'une réduction Geniu...,"8,1",15
871,La Rochelle,STUDIO EN HYPERCENTRE À 2 PAS DE L'HOTEL DE VI...,https://www.booking.com/hotel/fr/studio-en-hyp...,46.159836,-1.152469,\nLocated 800 metres from Concurrence and 2.4 ...,1.0,15
872,La Rochelle,"Hotel La Marine, Vieux Port",https://www.booking.com/hotel/fr/la-marine-la-...,46.158159,-1.152671,\nCet établissement est à 6 minutes à pied de ...,"8,5",15
873,La Rochelle,"Studio La Rochelle, 1 pièce, 2 personnes - FR-...",https://www.booking.com/hotel/fr/apartment-por...,46.139260,-1.160848,"\nLe Studio La Rochelle, 1 pièce, 2 personnes ...",1.0,15


In [28]:
test_meteo = pd.read_csv("df_gps_weather.csv")
test_meteo


,ville,lat,lon,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,score
0,Mont Saint Michel,48.635954,-1.511460,"[1, 2, 3, 4, 5, 6, 7]","[17, 20, 20, 17, 19, 21, 19]","[52, 0, 15, 62, 37, 0, 0]","['Rain', 'Clouds', 'Clouds', 'Rain', 'Rain', '...",16
1,St Malo,48.649518,-2.026041,"[1, 2, 3, 4, 5, 6, 7]","[15, 19, 18, 16, 16, 20, 17]","[47, 0, 9, 53, 48, 0, 0]","['Rain', 'Clouds', 'Clouds', 'Rain', 'Rain', '...",15
2,Bayeux,49.276462,-0.702474,"[1, 2, 3, 4, 5, 6, 7]","[17, 20, 21, 14, 19, 22, 20]","[65, 0, 38, 71, 46, 0, 0]","['Rain', 'Clouds', 'Clouds', 'Rain', 'Rain', '...",15
3,Le Havre,49.493898,0.107973,"[1, 2, 3, 4, 5, 6, 7]","[17, 18, 19, 14, 17, 18, 16]","[32, 0, 60, 98, 50, 0, 0]","['Clear', 'Clouds', 'Rain', 'Rain', 'Rain', 'C...",13
4,Rouen,49.440459,1.093966,"[1, 2, 3, 4, 5, 6, 7]","[19, 22, 22, 16, 18, 21, 21]","[24, 0, 18, 100, 70, 0, 0]","['Clear', 'Clouds', 'Clouds', 'Rain', 'Rain', ...",16
5,Paris,48.858890,2.320041,"[1, 2, 3, 4, 5, 6, 7]","[21, 23, 24, 22, 20, 23, 24]","[2, 0, 0, 81, 61, 0, 0]","['Clouds', 'Clouds', 'Clouds', 'Rain', 'Rain',...",20
6,Amiens,49.894171,2.295695,"[1, 2, 3, 4, 5, 6, 7]","[20, 22, 23, 15, 18, 21, 21]","[37, 0, 0, 96, 86, 0, 0]","['Clouds', 'Clear', 'Clouds', 'Rain', 'Rain', ...",16
7,Lille,50.636565,3.063528,"[1, 2, 3, 4, 5, 6, 7]","[20, 22, 24, 19, 20, 21, 22]","[57, 0, 0, 61, 94, 0, 0]","['Rain', 'Clear', 'Clouds', 'Rain', 'Rain', 'C...",18
8,Strasbourg,48.584614,7.750713,"[1, 2, 3, 4, 5, 6, 7]","[19, 23, 23, 29, 15, 24, 28]","[100, 16, 20, 21, 100, 20, 0]","['Rain', 'Clouds', 'Clouds', 'Clear', 'Rain', ...",19
9,Chateau du Haut Koenigsbourg,48.249523,7.345492,"[1, 2, 3, 4, 5, 6, 7]","[15, 20, 22, 26, 12, 21, 26]","[100, 2, 18, 60, 100, 20, 0]","['Rain', 'Clouds', 'Clouds', 'Clear', 'Rain', ...",16


## Create your data lake using S3

In [29]:
# Let's use Amazon S3
s3 = boto3.resource("s3")


2022-06-26 07:43:04 [botocore.credentials] INFO: Found credentials in shared credentials file: ~/.aws/credentials


In [30]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)


alexlasnier-bucket


In [31]:
bucket = s3.Bucket("alexlasnier-bucket")


In [32]:
bucket.upload_file("df_gps_weather.csv", "meteo.csv")
bucket.upload_file("df_gps_hotels.csv", "hotels.csv")


In [33]:
for object_summary in bucket.objects.filter():
    print(object_summary.key)


hotels.csv
meteo.csv


## ETL

In [34]:
df_gps_weather = pd.read_csv("s3://alexlasnier-bucket/meteo.csv")
df_gps_hotels = pd.read_csv("s3://alexlasnier-bucket/hotels.csv")


2022-06-26 07:43:06 [botocore.credentials] INFO: Found credentials in shared credentials file: ~/.aws/credentials


In [35]:
df_weather = pd.DataFrame()
# convert into list
df_weather[["ville", "lat", "lon", "score"]] = df_gps_weather[
    ["ville", "lat", "lon", "score"]
]


df_weather["jour_+x"] = df_gps_weather["jour_+x"].apply(eval)
df_weather["temperature_ressentie"] = df_gps_weather["temperature_ressentie"].apply(
    eval
)
df_weather["probabilite_de_pluie"] = df_gps_weather["probabilite_de_pluie"].apply(eval)
df_weather["meteo_principale"] = df_gps_weather["meteo_principale"].apply(eval)


In [36]:
df_gps_hotels


,ville,hotels,liens,lat,lon,description,score,meteo_score
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,48.614700,-1.509617,\nVous pouvez bénéficier d'une réduction Geniu...,"8,0",16
1,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.617587,-1.510396,\n\nVous pouvez bénéficier d'une réduction Gen...,"7,9",16
2,Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.616882,-1.510918,\nCet établissement est à 1 minute à pied de l...,"7,2",16
3,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.635349,-1.510379,\nVous pouvez bénéficier d'une réduction Geniu...,"7,3",16
4,Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,48.612938,-1.510105,"\nNiché dans un écrin de verdure, à seulement ...","7,2",16
...,...,...,...,...,...,...,...,...
870,La Rochelle,Hôtel La Tour de Nesle La Rochelle Vieux Port,https://www.booking.com/hotel/fr/de-la-tour-de...,46.158097,-1.150362,\nVous pouvez bénéficier d'une réduction Geniu...,"8,1",15
871,La Rochelle,STUDIO EN HYPERCENTRE À 2 PAS DE L'HOTEL DE VI...,https://www.booking.com/hotel/fr/studio-en-hyp...,46.159836,-1.152469,\nLocated 800 metres from Concurrence and 2.4 ...,1.0,15
872,La Rochelle,"Hotel La Marine, Vieux Port",https://www.booking.com/hotel/fr/la-marine-la-...,46.158159,-1.152671,\nCet établissement est à 6 minutes à pied de ...,"8,5",15
873,La Rochelle,"Studio La Rochelle, 1 pièce, 2 personnes - FR-...",https://www.booking.com/hotel/fr/apartment-por...,46.139260,-1.160848,"\nLe Studio La Rochelle, 1 pièce, 2 personnes ...",1.0,15


In [37]:
df_weather[
    ["Temp_1", "Temp_2", "Temp_3", "Temp_4", "Temp_5", "Temp_6", "Temp_7"]
] = pd.DataFrame(df_weather["temperature_ressentie"].to_list(), index=df_weather.index)
df_weather["Temp_moyenne_7jours"] = (
    df_weather[["Temp_1", "Temp_2", "Temp_3", "Temp_4", "Temp_5", "Temp_6", "Temp_7"]]
    .mean(axis=1)
    .round(4)
)
df_weather = df_weather.drop(
    columns=["Temp_1", "Temp_2", "Temp_3", "Temp_4", "Temp_5", "Temp_6", "Temp_7"]
)


In [38]:
df_weather[
    ["Temp_1", "Temp_2", "Temp_3", "Temp_4", "Temp_5", "Temp_6", "Temp_7"]
] = pd.DataFrame(df_weather["probabilite_de_pluie"].to_list(), index=df_weather.index)
df_weather["Proba_pluie_7jours"] = (
    df_weather[["Temp_1", "Temp_2", "Temp_3", "Temp_4", "Temp_5", "Temp_6", "Temp_7"]]
    .mean(axis=1)
    .round(4)
)
df_weather = df_weather.drop(
    columns=["Temp_1", "Temp_2", "Temp_3", "Temp_4", "Temp_5", "Temp_6", "Temp_7"]
)


In [39]:
# Calculate a scale to reflect the difference between
# the min and max of the avg temperature (not the more effective way I think)
#
# first, calculate a ratio between max and min and divide it to have 17.5 steps (arbitrary)

temp_diff = (
    df_weather["Temp_moyenne_7jours"].max() - df_weather["Temp_moyenne_7jours"].min()
) / 17.5

# calculate the scale value by subtracting the min value from the avg temperature, divide
# that by the ratio which will give the scale a value between 0...16.5 and add 1 to it so that
# the scale values start at 1 (to be visible on the map)
# add the according scale to each row
# the scale column will then be used for size=... parameter in the scatter_mapbox call below
df_weather["scale_temp"] = (
    df_weather["Temp_moyenne_7jours"] - df_weather["Temp_moyenne_7jours"].min()
) / temp_diff + 1


In [40]:
proba_diff = (
    df_weather["Proba_pluie_7jours"].max() - df_weather["Proba_pluie_7jours"].min()
) / 17.5

df_weather["scale_pluie"] = (
    df_weather["Proba_pluie_7jours"] - df_weather["Proba_pluie_7jours"].min()
) / proba_diff + 1


### Connections

In [41]:
DBHOST = "alexlasnier-db.cma6tegzy28v.eu-west-3.rds.amazonaws.com"
PORT = 5432
DBNAME = "postgres"
DBUSER = "alexlasnieruser"
DBPASS = "!CitroenC3DaciaLogan"


In [42]:
# Create engine will create a connection between a SQLlite DB and python
engine = create_engine(
    f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}", echo=True
).connect()

# Initialize a sessionmaker
Session = sessionmaker(bind=engine)

# Instanciate Session
session = Session()


2022-06-26 07:43:07,981 INFO sqlalchemy.engine.Engine select pg_catalog.version()


2022-06-26 07:43:07 [sqlalchemy.engine.Engine] INFO: select pg_catalog.version()


2022-06-26 07:43:07,982 INFO sqlalchemy.engine.Engine [raw sql] {}


2022-06-26 07:43:07 [sqlalchemy.engine.Engine] INFO: [raw sql] {}


2022-06-26 07:43:08,012 INFO sqlalchemy.engine.Engine select current_schema()


...

2022-06-26 07:43:08 [sqlalchemy.engine.Engine] INFO: [raw sql] {}


2022-06-26 07:43:08,037 INFO sqlalchemy.engine.Engine show standard_conforming_strings


2022-06-26 07:43:08 [sqlalchemy.engine.Engine] INFO: show standard_conforming_strings


2022-06-26 07:43:08,038 INFO sqlalchemy.engine.Engine [raw sql] {}


2022-06-26 07:43:08 [sqlalchemy.engine.Engine] INFO: [raw sql] {}


### Store in RDS

In [43]:
df_weather.to_sql("meteo", engine, if_exists="replace", index=False)
df_gps_hotels.to_sql("hotels", engine, if_exists="replace", index=False)


2022-06-26 07:43:08,084 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:08 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:08,085 INFO sqlalchemy.engine.Engine [generated in 0.00108s] {'name': 'meteo'}


2022-06-26 07:43:08 [sqlalchemy.engine.Engine] INFO: [generated in 0.00108s] {'name': 'meteo'}


2022-06-26 07:43:08,111 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


...

2022-06-26 07:43:08,739 INFO sqlalchemy.engine.Engine [generated in 0.00351s] ({'ville': 'Mont Saint Michel', 'hotels': 'Hôtel Vert', 'liens': 'https://www.booking.com/hotel/fr/vert.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaE2IAQGYAQ24AQbIAQzYAQHoAQH4AQKIAgGoAgO4AuDI35UGwAIB0gIkMTA2N2I3ODItMW ... (135 characters truncated) ... no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=1d9f2172522f009b&srepoch=1656218726&from=searchresults#hotelTmpl', 'lat': 48.61470049, 'lon': -1.50961697, 'description': "\nVous pouvez bénéficier d'une réduction Genius dans l'établissement Hôtel Vert\xa0! Connectez-vous pour économiser.\nSitué à 2 km du Mont-Saint-Mich ... (302 characters truncated) ... se privée.\nLe Vert Hotel possède également une réception ouverte 24h/24, un parking privé, un supermarché sur place et une boutique de souvenirs. \n", 'score': '8,0', 'meteo_score': 16}, {'ville': 'Mont Saint Michel', 'hotels': 'Le Relais Saint Michel', 'liens': 'https://www.b

2022-06-26 07:43:08 [sqlalchemy.engine.Engine] INFO: [generated in 0.00351s] ({'ville': 'Mont Saint Michel', 'hotels': 'Hôtel Vert', 'liens': 'https://www.booking.com/hotel/fr/vert.fr.html?label=gen173nr-1FCAEoggI46AdIDVgEaE2IAQGYAQ24AQbIAQzYAQHoAQH4AQKIAgGoAgO4AuDI35UGwAIB0gIkMTA2N2I3ODItMW ... (135 characters truncated) ... no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=1d9f2172522f009b&srepoch=1656218726&from=searchresults#hotelTmpl', 'lat': 48.61470049, 'lon': -1.50961697, 'description': "\nVous pouvez bénéficier d'une réduction Genius dans l'établissement Hôtel Vert\xa0! Connectez-vous pour économiser.\nSitué à 2 km du Mont-Saint-Mich ... (302 characters truncated) ... se privée.\nLe Vert Hotel possède également une réception ouverte 24h/24, un parking privé, un supermarché sur place et une boutique de souvenirs. \n", 'score': '8,0', 'meteo_score': 16}, {'ville': 'Mont Saint Michel', 'hotels': 'Le Relais Saint Michel', 'liens': 'https://www.bo

2022-06-26 07:43:10,635 INFO sqlalchemy.engine.Engine COMMIT


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: COMMIT


875

In [44]:
# Test SQL request to get data from RDS
query_meteo = text("SELECT * FROM meteo")
df_query_meteo = pd.read_sql(query_meteo, engine)
df_query_meteo


2022-06-26 07:43:10,678 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:10,679 INFO sqlalchemy.engine.Engine [cached since 2.596s ago] {'name': 'SELECT * FROM meteo'}


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: [cached since 2.596s ago] {'name': 'SELECT * FROM meteo'}


2022-06-26 07:43:10,704 INFO sqlalchemy.engine.Engine SELECT * FROM meteo


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: SELECT * FROM meteo


2022-06-26 07:43:10,704 INFO sqlalchemy.engine.Engine [generated in 0.00069s] {}


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: [generated in 0.00069s] {}


,ville,lat,lon,score,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,Temp_moyenne_7jours,Proba_pluie_7jours,scale_temp,scale_pluie
0,Mont Saint Michel,48.635954,-1.511460,16,"{1,2,3,4,5,6,7}","{17,20,20,17,19,21,19}","{52,0,15,62,37,0,0}","{Rain,Clouds,Clouds,Rain,Rain,Clouds,Clear}",19.0000,23.7143,3.752806,4.979757
1,St Malo,48.649518,-2.026041,15,"{1,2,3,4,5,6,7}","{15,19,18,16,16,20,17}","{47,0,9,53,48,0,0}","{Rain,Clouds,Clouds,Rain,Rain,Clouds,Clouds}",17.2857,22.4286,1.393238,4.489109
2,Bayeux,49.276462,-0.702474,15,"{1,2,3,4,5,6,7}","{17,20,21,14,19,22,20}","{65,0,38,71,46,0,0}","{Rain,Clouds,Clouds,Rain,Rain,Clouds,Clouds}",19.0000,31.4286,3.752806,7.923684
3,Le Havre,49.493898,0.107973,13,"{1,2,3,4,5,6,7}","{17,18,19,14,17,18,16}","{32,0,60,98,50,0,0}","{Clear,Clouds,Rain,Rain,Rain,Clouds,Clear}",17.0000,34.2857,1.000000,9.014009
4,Rouen,49.440459,1.093966,16,"{1,2,3,4,5,6,7}","{19,22,22,16,18,21,21}","{24,0,18,100,70,0,0}","{Clear,Clouds,Clouds,Rain,Rain,Clouds,Clear}",19.8571,30.2857,4.932521,7.487531
5,Paris,48.858890,2.320041,20,"{1,2,3,4,5,6,7}","{21,23,24,22,20,23,24}","{2,0,0,81,61,0,0}","{Clouds,Clouds,Clouds,Rain,Rain,Clouds,Clear}",22.4286,20.5714,8.471941,3.780365
6,Amiens,49.894171,2.295695,16,"{1,2,3,4,5,6,7}","{20,22,23,15,18,21,21}","{37,0,0,96,86,0,0}","{Clouds,Clear,Clouds,Rain,Rain,Clouds,Clear}",20.0000,31.2857,5.129209,7.869150
7,Lille,50.636565,3.063528,18,"{1,2,3,4,5,6,7}","{20,22,24,19,20,21,22}","{57,0,0,61,94,0,0}","{Rain,Clear,Clouds,Rain,Rain,Clouds,Clouds}",21.1429,30.2857,6.702300,7.487531
8,Strasbourg,48.584614,7.750713,19,"{1,2,3,4,5,6,7}","{19,23,23,29,15,24,28}","{100,16,20,21,100,20,0}","{Rain,Clouds,Clouds,Clear,Rain,Rain,Clear}",23.0000,39.5714,9.258418,11.031135
9,Chateau du Haut Koenigsbourg,48.249523,7.345492,16,"{1,2,3,4,5,6,7}","{15,20,22,26,12,21,26}","{100,2,18,60,100,20,0}","{Rain,Clouds,Clouds,Clear,Rain,Rain,Clear}",20.2857,42.8571,5.522447,12.285022


In [45]:
# Test SQL request to get data from RDS
query_hotels = text("SELECT * FROM hotels ")
df_query_hotels = pd.read_sql(query_hotels, engine)
df_query_hotels


2022-06-26 07:43:10,762 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:10,763 INFO sqlalchemy.engine.Engine [cached since 2.68s ago] {'name': 'SELECT * FROM hotels '}


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: [cached since 2.68s ago] {'name': 'SELECT * FROM hotels '}


2022-06-26 07:43:10,778 INFO sqlalchemy.engine.Engine SELECT * FROM hotels 


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: SELECT * FROM hotels 


2022-06-26 07:43:10,779 INFO sqlalchemy.engine.Engine [generated in 0.00078s] {}


2022-06-26 07:43:10 [sqlalchemy.engine.Engine] INFO: [generated in 0.00078s] {}


,ville,hotels,liens,lat,lon,description,score,meteo_score
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,48.614700,-1.509617,\nVous pouvez bénéficier d'une réduction Geniu...,"8,0",16
1,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,48.617587,-1.510396,\n\nVous pouvez bénéficier d'une réduction Gen...,"7,9",16
2,Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,48.616882,-1.510918,\nCet établissement est à 1 minute à pied de l...,"7,2",16
3,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,48.635349,-1.510379,\nVous pouvez bénéficier d'une réduction Geniu...,"7,3",16
4,Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,48.612938,-1.510105,"\nNiché dans un écrin de verdure, à seulement ...","7,2",16
...,...,...,...,...,...,...,...,...
870,La Rochelle,Résidence Pierre & Vacances Centre,https://www.booking.com/hotel/fr/pierre-vacanc...,46.155769,-1.146392,\nLa Résidence Pierre & Vacances Centre propos...,"7,3",15
871,La Rochelle,Hôtel La Tour de Nesle La Rochelle Vieux Port,https://www.booking.com/hotel/fr/de-la-tour-de...,46.158097,-1.150362,\nVous pouvez bénéficier d'une réduction Geniu...,"8,1",15
872,La Rochelle,"Hotel La Marine, Vieux Port",https://www.booking.com/hotel/fr/la-marine-la-...,46.158159,-1.152671,\nCet établissement est à 6 minutes à pied de ...,"8,5",15
873,La Rochelle,"Studio La Rochelle, 1 pièce, 2 personnes - FR-...",https://www.booking.com/hotel/fr/apartment-por...,46.139260,-1.160848,"\nLe Studio La Rochelle, 1 pièce, 2 personnes ...",1.0,15


### Visualization

In [46]:
fig = px.scatter_mapbox(
    df_query_meteo,
    lat="lat",
    lon="lon",
    hover_name="ville",
    hover_data={
        "ville": True,
        "lat": False,
        "lon": False,
        "jour_+x": False,
        "temperature_ressentie": False,
        "probabilite_de_pluie": False,
        "meteo_principale": True,
        "Temp_moyenne_7jours": True,
        "Proba_pluie_7jours": True,
        "score": True,
        "scale_temp": False,
        "scale_pluie": False,
    },
    color="Temp_moyenne_7jours",
    size="scale_temp",
    zoom=4,
    color_continuous_scale="thermal",
    mapbox_style="carto-positron",
)
fig.show()


In [47]:
fig = px.scatter_mapbox(
    df_query_meteo,
    lat="lat",
    lon="lon",
    hover_name="ville",
    hover_data={
        "ville": True,
        "lat": False,
        "lon": False,
        "jour_+x": False,
        "temperature_ressentie": False,
        "probabilite_de_pluie": False,
        "meteo_principale": True,
        "Temp_moyenne_7jours": True,
        "Proba_pluie_7jours": True,
        "score": True,
        "scale_temp": False,
        "scale_pluie": False,
    },
    color="Proba_pluie_7jours",
    size="scale_pluie",
    zoom=4,
    color_continuous_scale="deep",
    mapbox_style="carto-positron",
)
fig.show()


## Top-5 destinations

In [48]:
top5_cities = text("SELECT * FROM meteo ORDER BY score DESC LIMIT 5")
df_top5 = pd.read_sql(top5_cities, engine)
df_top5


2022-06-26 07:43:11,226 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:11 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:11,227 INFO sqlalchemy.engine.Engine [cached since 3.144s ago] {'name': 'SELECT * FROM meteo ORDER BY score DESC LIMIT 5'}


2022-06-26 07:43:11 [sqlalchemy.engine.Engine] INFO: [cached since 3.144s ago] {'name': 'SELECT * FROM meteo ORDER BY score DESC LIMIT 5'}


2022-06-26 07:43:11,240 INFO sqlalchemy.engine.Engine SELECT * FROM meteo ORDER BY score DESC LIMIT 5


2022-06-26 07:43:11 [sqlalchemy.engine.Engine] INFO: SELECT * FROM meteo ORDER BY score DESC LIMIT 5


2022-06-26 07:43:11,240 INFO sqlalchemy.engine.Engine [generated in 0.00074s] {}


2022-06-26 07:43:11 [sqlalchemy.engine.Engine] INFO: [generated in 0.00074s] {}


,ville,lat,lon,score,jour_+x,temperature_ressentie,probabilite_de_pluie,meteo_principale,Temp_moyenne_7jours,Proba_pluie_7jours,scale_temp,scale_pluie
0,Nimes,43.837425,4.360069,27,"{1,2,3,4,5,6,7}","{29,27,29,29,28,29,34}","{11,95,0,0,1,0,0}","{Clouds,Rain,Clear,Clouds,Clear,Clear,Clear}",29.2857,15.2857,17.910074,1.763239
1,Avignon,43.949249,4.805901,27,"{1,2,3,4,5,6,7}","{29,25,29,31,27,28,34}","{15,99,0,2,0,0,0}","{Clouds,Rain,Clear,Clouds,Clear,Clear,Clear}",29.0000,16.5714,17.516835,2.253887
2,Aix en Provence,43.529842,5.447474,27,"{1,2,3,4,5,6,7}","{31,28,29,29,29,29,33}","{28,76,0,40,0,0,0}","{Clouds,Rain,Clear,Rain,Clear,Clear,Clear}",29.7143,20.5714,18.500000,3.780365
3,Uzes,44.012128,4.419672,27,"{1,2,3,4,5,6,7}","{28,26,30,30,27,29,34}","{15,96,0,0,1,0,0}","{Clouds,Rain,Clear,Clouds,Clear,Clear,Clear}",29.1429,16.0000,17.713523,2.035830
4,Cassis,43.214036,5.539632,26,"{1,2,3,4,5,6,7}","{29,26,28,26,28,28,30}","{10,67,0,45,0,0,0}","{Clouds,Rain,Clear,Rain,Clear,Clear,Clear}",27.8571,17.4286,15.943744,2.581011


In [49]:
fig = px.scatter_mapbox(
    df_top5,
    lat="lat",
    lon="lon",
    hover_name="ville",
    hover_data={
        "ville": True,
        "lat": False,
        "lon": False,
        "jour_+x": False,
        "temperature_ressentie": False,
        "probabilite_de_pluie": False,
        "meteo_principale": True,
        "Temp_moyenne_7jours": True,
        "Proba_pluie_7jours": True,
        "score": True,
        "scale_temp": False,
        "scale_pluie": False,
    },
    color="Proba_pluie_7jours",
    size="scale_pluie",
    zoom=5,
    color_continuous_scale="deep",
    mapbox_style="carto-positron",
)
fig.show()


## Top-20 hotels in the area

In [50]:
top20_hotels = text(
    "SELECT * FROM hotels WHERE ville IN (SELECT ville FROM meteo ORDER BY score DESC LIMIT 5) ORDER BY score DESC LIMIT 20"
)


# need to nested query to get the score of the hotel => slect * form hotels where (query top5_cities) limit 20

# Et du coup pas besoin de score meteo dans df_gps_hotels...

df_top20 = pd.read_sql(top20_hotels, engine)
df_top20


2022-06-26 07:43:11,341 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:11 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-06-26 07:43:11,342 INFO sqlalchemy.engine.Engine [cached since 3.258s ago] {'name': 'SELECT * FROM hotels WHERE ville IN (SELECT ville FROM meteo ORDER BY score DESC LIMIT 5) ORDER BY score DESC LIMIT 20'}


2022-06-26 07:43:11 [sqlalchemy.engine.Engine] INFO: [cached since 3.258s ago] {'name': 'SELECT * FROM hotels WHERE ville IN (SELECT ville FROM meteo ORDER BY score DESC LIMIT 5) ORDER BY score DESC LIMIT 20'}


2022-06-26 07:43:11,356 INFO sqlalchemy.engine.Engine SELECT * FROM hotels WHERE ville IN (SELECT ville FROM meteo ORDER BY score DESC LIMIT 5) ORDER BY score DESC LIMIT 20


2022-06-26 07:43:11 [sqlalchemy.engine.Engine] INFO: SELECT * FROM hotels WHERE ville IN (SELECT ville FROM meteo ORDER BY score DESC LIMIT 5) ORDER BY score DESC LIMIT 20


2022-06-26 07:43:11,357 INFO sqlalchemy.engine.Engine [generated in 0.00081s] {}


2022-06-26 07:43:11 [sqlalchemy.engine.Engine] INFO: [generated in 0.00081s] {}


,ville,hotels,liens,lat,lon,description,score,meteo_score
0,Uzes,AU BORD DE L'ALZON,https://www.booking.com/hotel/fr/au-bord-de-l-...,43.995238,4.421976,\n\nVous pouvez bénéficier d'une réduction Gen...,"9,9",27
1,Uzes,Mas d orient,https://www.booking.com/hotel/fr/mas-d-orient-...,44.020244,4.409254,\n\nVous pouvez bénéficier d'une réduction Gen...,"9,8",27
2,Uzes,Appartement de charme dans le cœur historique ...,https://www.booking.com/hotel/fr/appartement-d...,44.012505,4.420930,\nVous pouvez bénéficier d'une réduction Geniu...,"9,7",27
3,Uzes,"Maison de charme Uzès, parking privé",https://www.booking.com/hotel/fr/maison-de-cha...,44.014756,4.420526,"\nSituée à Uzès, en Languedoc-Roussillon, la M...","9,7",27
4,Uzes,DUPLEX CENTRE HISTORIQUE UZES,https://www.booking.com/hotel/fr/duplex-centre...,44.012349,4.418470,\n\nVous pouvez bénéficier d'une réduction Gen...,"9,7",27
5,Cassis,SunSet Cassis,https://www.booking.com/hotel/fr/sunset-cassis...,43.215456,5.535734,\nCet établissement est à 1 minute à pied de l...,"9,5",26
6,Cassis,SunRise Cassis,https://www.booking.com/hotel/fr/sunrise-cassi...,43.215219,5.535682,\nCet établissement est à 2 minutes à pied de ...,"9,5",26
7,Avignon,"la chambre des petits papes, intra muros",https://www.booking.com/hotel/fr/la-chambre-de...,43.951352,4.811561,"\nSituée à Avignon, la chambre des petits pape...","9,5",27
8,Uzes,La petite maison,https://www.booking.com/hotel/fr/la-petite-mai...,44.013512,4.416768,\nLa petite maison est située à Uzès. Offrant ...,"9,3",27
9,Cassis,La douceur de Cassis,https://www.booking.com/hotel/fr/la-douceur-de...,43.215009,5.530956,\nCet établissement est à 6 minutes à pied de ...,"9,3",26


In [54]:
fig = px.scatter_mapbox(
    df_top20,
    lat="lat",
    lon="lon",
    hover_name="hotels",
    hover_data={
        "ville": True,
        "lat": False,
        "lon": False,
        "score": True,
    },
    color="ville",
    zoom=6,
    mapbox_style="carto-positron",
)
fig.show()
